In [18]:
###
### Fetch full item catalog
###

# format:
# "id":9,
# "roblox_assetid":21070012,
# "roblox_rap":13577229,
# "roblox_value":20000000,
# "robux_volume_30d_total":0,
# "popularity_position":2499,
# "is_valued":true,
# "is_rare":true,
# "is_projected":false,
# "roblox_name":"Dominus Empyreus",
# "roblox_acronym":"Emp",
# "roblox_lowestprice":"600000000",
# "price_updated_at":"2023-04-29T18:23:17.345959Z",
# "went_limited_at":"2010-01-23T01:25:00.407Z",
# "image":{"420":"https://tr.rbxcdn.com/b34f3766b22d7db843c8252000b4f555/420/420/Hat/Png",
# "150":"https://tr.rbxcdn.com/b34f3766b22d7db843c8252000b4f555/150/150/Hat/Png",
# "30":"https://tr.rbxcdn.com/b34f3766b22d7db843c8252000b4f555/30/30/Hat/Png"}

import requests
import json

# NOTE: This is not the entire Roblox catalog. It is just the items with limited quantity.
# These are the only items which actually carry IRL value. (Scarcity = value)
# Limiteds can be official Roblox gear or they can be UGC (user generated content).
url = "https://rblx.trade/api/v1/catalog/all"
dir = "./roblox-catalog.json"
try:
    file = open(dir, "r")
    print("catalog is cached")
    catalog = json.loads(file.read())
    file.close()
except:
    catalog_req = requests.get(url)
    print("no cache found at " + url + "; sending request")
    print("status: " + str(catalog_req.status_code))
    catalog = json.loads(catalog_req.text)
    file = open(dir, "w")
    file.write(catalog_req.text)
    file.close()

print("items found: " + str(len(catalog)))

# clustering -> vectoring -> graph euclidian distance over points on x-y plots -> check out smallest vector distances
# used in AI - see vector databases
# vector databases take the vector fields and store them, then do the calcs. used during ai training process
# visualize vector databases
# vector database as data source -> gradient output, heatmap? 
# track over time - which had the highest changes?

catalog is cached
items found: 2498


In [17]:
###
### Fetch asset prices and volumes
###

# format:
# [{
#    average_price: Number,     // in robux
#    volume: Number,            // number of trades,
#    date: Number,              // unix timestamp
# }]

import os
import requests
import time

item_history_by_id = None
dir = "./roblox-itemvalues/"
try:
    os.mkdir(dir)
except:
    None
history_api = "https://rblx.trade/api/v1/item/graph/price-and-volume?assetId="

def get_history(asset_id):
    filepath = os.path.join(dir, str(asset_id) + ".json")
    try:
        file = open(filepath, "r")
        print("SKIPPED #"+str(asset_id)+" (in cache)", end="\33[2K\r")
        file.close()
    except:
        file = open(filepath, "w")
        fetched = False
        while(not fetched):
            try:
                history_req = requests.get(history_api + str(asset_id), timeout=2)
                fetched = True
            except:
                time.sleep(2)
        file.write(history_req.text)
        file.close()

# i hope there's a better way to do this
count = len(catalog["roblox_assetid"])
i = 0
for asset_id in catalog["roblox_assetid"]:
    i += 1
    print("fetching "+str(asset_id)+": "+str(i)+"/"+str(count), end="\33[2K\r")
    get_history(asset_id)
file.close()

In [ ]:
###
### Massage data into pandas
###

import pandas

# Some metrics to encapsulate the historical data

total_lifetime_volume = None # sum all the volumes
total_historical_datapoints = None # len(arr)
mean_daily_trades = None
mean_daily_trades_30days = None
mean_daily_trades_1yr = None
# some sort of trend? 

df = pandas.DataFrame(catalog)
df.drop(columns=["image"], inplace=True)

In [19]:
###
### Visualize!
###

import perspective

table = perspective.Table(df)
perspective.PerspectiveWidget(table)

PerspectiveWidget(columns=['index', 'id', 'roblox_assetid', 'roblox_rap', 'roblox_value', 'robux_volume_30d_to…